# Film-KG: RDF/SPARQL Regeln materialisieren

Dieses Notebook lädt deine Tripeldatei, erstellt zwei Regeln und materialisiert die Kanten:

1. **SAME_UNIVERSE**: Zwei Filme teilen sich (kanonisch) mindestens eine Figur → `:SAME_UNIVERSE`  
2. **CREATIVE_PAIR (Director×Actor)**: Regisseur und Schauspieler haben ≥2 gemeinsame Filme → `:CREATIVE_PAIR {roles: ["Director","Actor"], co_works:n}`

> **Hinweis:** Passe ggf. den Pfad zu deiner Tripeldatei an. Standardmäßig wird `/mnt/data/movie_kg_triples.tsv` erwartet (Subject \t Predicate \t Object).


In [3]:
# (1) Setup: Bibliotheken importieren
!python -c "import rdflib" 2>/dev/null || pip -q install rdflib==7.0.0

from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD
from pathlib import Path

DATA_PATH = Path("../data/kg/triples/movie_kg_triples.tsv")  # ggf. anpassen
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True, parents=True)

print("Benutze Datendatei:", DATA_PATH.resolve())
print("Output-Verzeichnis:", OUT_DIR.resolve())


Benutze Datendatei: /Users/tschaffel/PycharmProjects/letterboxd-KG/data/kg/triples/movie_kg_triples.tsv
Output-Verzeichnis: /Users/tschaffel/PycharmProjects/letterboxd-KG/data/kg/outputs


In [6]:
# (2) Tripel laden (TSV: S\tP\tO) in einen RDFLib-Graphen
g = Graph()

SCHEMA = Namespace("http://schema.org/")
EX     = Namespace("http://example.org/")
g.bind("schema", SCHEMA)
g.bind("ex", EX)
g.bind("rdf", RDF)

prefix_map = {
    "schema": str(SCHEMA),
    "rdf": str(RDF),
    "rdfs": str(RDFS),
    "xsd": str(XSD),
}

from rdflib import URIRef, Literal

def parse_term(term: str):
    term = term.strip()
    # 1) explizites Literal
    if len(term) >= 2 and term[0] == '"' and term[-1] == '"':
        return Literal(term[1:-1])
    # 2) absolute URI
    if term.startswith("http://") or term.startswith("https://"):
        return URIRef(term)
    # 3) QName
    if ":" in term:
        pfx, local = term.split(":", 1)
        if pfx in prefix_map:
            return URIRef(prefix_map[pfx] + local)
    # 4) sonst als Literal (z.B. Personenname ohne Quotes)
    return Literal(term)

count = 0
with DATA_PATH.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        parts = line.split("\t")
        if len(parts) != 3:
            continue
        s, p, o = map(parse_term, parts)
        g.add((s, p, o))
        count += 1

print(f"Geladene Tripel: {count}")
print("Beispiel-Tripel:")
for i, (s,p,o) in enumerate(g):
    print("-", s, p, o)
    if i >= 4:
        break


Geladene Tripel: 58601
Beispiel-Tripel:
- movie496 http://schema.org/actor person1123205
- person52366 http://schema.org/name Andrew Bryniarski
- company27711 ex:country US
- movie19164 http://schema.org/genre genre12
- movie492719 http://schema.org/character Yuko Tani (voice)


## Regel 1: `:SAME_UNIVERSE` über geteilte Figur (mit Kanonisierung)

In [7]:
# (3) SAME_UNIVERSE konstruieren
same_universe_q = 'PREFIX schema: <http://schema.org/>\nPREFIX ex:     <http://kg.local/>\n\nCONSTRUCT {\n  ?f1 ex:SAME_UNIVERSE ?f2 .\n}\nWHERE {\n  ?f1 schema:character ?ch1 .\n  ?f2 schema:character ?ch2 .\n  FILTER(?f1 != ?f2)\n\n  # Kanonisierung: Klammerzusätze entfernen (z.B. "(voice)"), trimmen, lower-case\n  BIND( LCASE(STR(REPLACE(STR(?ch1), "\\\\s*\\\\([^)]*\\\\)", ""))) AS ?canon1 )\n  BIND( LCASE(STR(REPLACE(STR(?ch2), "\\\\s*\\\\([^)]*\\\\)", ""))) AS ?canon2 )\n\n  FILTER(STRLEN(STR(?canon1)) > 0)\n  FILTER(?canon1 = ?canon2)\n}\n'

su_graph = g.query(same_universe_q).graph
print("SAME_UNIVERSE-Kanten:", len(su_graph))
su_path = OUT_DIR / "derived_same_universe.ttl"
su_graph.serialize(destination=str(su_path), format="turtle")
print("Gespeichert:", su_path.resolve())

# Optional: in den Hauptgraphen materialisieren
for t in su_graph:
    g.add(t)

print("Gesamttripel nach Materialisierung:", len(g))


KeyboardInterrupt: 

## Regel 2: `:CREATIVE_PAIR` (Director × Actor, ≥2 gemeinsame Filme)

In [ ]:
creative_pair_q = 'PREFIX schema: <http://schema.org/>\nPREFIX ex:     <http://kg.local/>\n\nCONSTRUCT {\n  ?d ex:CREATIVE_PAIR ?a .\n  ?d ex:creativePairRoles "Director,Actor" .\n  ?d ex:creativePairCount ?n .\n}\nWHERE {\n  SELECT ?d ?a (COUNT(DISTINCT ?f) AS ?n)\n  WHERE {\n    ?f schema:director ?d .\n    ?f schema:actor    ?a .\n  }\n  GROUP BY ?d ?a\n  HAVING (?n >= 2)\n}\n'
cp_graph = g.query(creative_pair_q).graph
cp_edges = sum(1 for (s,p,o) in cp_graph if str(p).endswith("CREATIVE_PAIR"))
print("CREATIVE_PAIR-Kanten:", cp_edges)
cp_path = OUT_DIR / "derived_creative_pair.ttl"
cp_graph.serialize(destination=str(cp_path), format="turtle")
print("Gespeichert:", cp_path.resolve())

# Optional: Materialisieren in Hauptgraph
for t in cp_graph:
    g.add(t)

print("Gesamttripel nach Materialisierung:", len(g))


## (Optional) Transitive Exploration über `:SAME_UNIVERSE`

In [ ]:
demo_q = 'PREFIX ex:     <http://kg.local/>\nSELECT DISTINCT ?g WHERE {\n  VALUES ?seed { ex:movie123 }    # <--- anpassen!\n  ?seed ex:SAME_UNIVERSE+ ?g .\n}\nLIMIT 50\n'
try:
    for row in g.query(demo_q):
        print(row)
except Exception as e:
    print("Hinweis: Passe zuerst den Seed an. Fehler:", e)


## Ergebnisse speichern

In [ ]:
all_path = OUT_DIR / "graph_with_derived.ttl"
g.serialize(destination=str(all_path), format="turtle")
print("Gesamter Graph inkl. abgeleiteter Kanten gespeichert nach:", all_path.resolve())


### Hinweise
- Für stabilere Kanten: Notebook regelmäßig (z. B. nightly) laufen lassen.
- Für Debugbarkeit: Zusätzliche Metadaten an Knoten anhängen (z. B. `creativePairRoles`, `creativePairCount`). Edge-Eigenschaften sind in RDF nicht nativ; Alternativen: Reifikation, RDF* oder Named Graphs.
- Falls du später `schema:composer` importierst, lässt sich die CREATIVE_PAIR-Regel 1:1 auf (Director×Composer) übertragen.
